# Import

In [1]:
import pandas
from rep.estimators import TMVAClassifier
import evaluation

ImportError: No module named rep.estimators

# Read training data

In [ ]:
folder = 'tau_data/'
train = pandas.read_csv(folder + 'training.csv', index_col='id')

In [ ]:
train.head()

# Define training features

In [ ]:
variables = ['LifeTime',
             'FlightDistance',
             'pt',
             ]

# TMVA baseline training

In [ ]:
baseline = TMVAClassifier(method='kBDT', features=variables, BoostType='Grad', 
                          NTrees=40, Shrinkage=0.01, MaxDepth=7,
                          UseNvars=6, nCuts=-1, MinNodeSize=10) 
baseline.fit(train, train['signal'])

# Check agreement test

In [ ]:
check_agreement = pandas.read_csv(folder + 'check_agreement.csv', index_col='id')
agreement_probs = baseline.predict_proba(check_agreement[variables])[:, 1]

ks = evaluation.compute_ks(
    agreement_probs[check_agreement['signal'].values == 0],
    agreement_probs[check_agreement['signal'].values == 1],
    check_agreement[check_agreement['signal'] == 0]['weight'].values,
    check_agreement[check_agreement['signal'] == 1]['weight'].values)
print 'KS metric', ks, ks < 0.09

# Check correlation test

In [ ]:
check_correlation = pandas.read_csv(folder + 'check_correlation.csv', index_col='id')
correlation_probs = baseline.predict_proba(check_correlation[variables])[:, 1]
cvm = evaluation.compute_cvm(correlation_probs, check_correlation['mass'])
print 'CvM metric', cvm, cvm < 0.002

# Compute weighted AUC on the training data with min_ANNmuon > 0.4

In [ ]:
train_eval = train[train['min_ANNmuon'] > 0.4]
train_probs = baseline.predict_proba(train_eval[variables])[:, 1]
AUC = evaluation.roc_auc_truncated(train_eval['signal'], train_probs)
print 'AUC', AUC

# Predict test, create file for kaggle

In [ ]:
test = pandas.read_csv(folder + 'test.csv', index_col='id')
result = pandas.DataFrame({'id': test.index})
result['prediction'] = baseline.predict_proba(test[variables])[:, 1]

In [ ]:
result.to_csv('baseline_tmva.csv', index=False, sep=',')